(a xor b) and (b xor c)

In [8]:
import keras.src.callbacks as cb
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
import tensorflow as tf
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import seaborn as sns

In [9]:
# Функция создания модели нейронной сети
def create_model() -> Sequential:
    """
    Creates a neural network model for binary classification.

    The model consists of 3 dense layers with ReLU activation function
    and sigmoid activation function in the output layer. It is compiled
    with binary cross-entropy loss, Adam optimizer and accuracy metric.

    Args:
        None

    Returns:
        Sequential: A compiled Keras neural network model for binary classification.

    Examples:
        >>> from tensorflow.keras.models import Sequential
        >>> baseline_model = create_model()
        >>> print(baseline_model.summary())
    """
    baseline_model: Sequential = Sequential()
    baseline_model.add(Input(shape=(3,), name='input'))
    baseline_model.add(Dense(1, activation='sigmoid', name='output'))

    baseline_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    baseline_model.summary()
    return baseline_model

In [10]:
def create_error_report(model_report: Sequential, val_split: float=0.2):
    data: pd.DataFrame = pd.read_csv('data/data_bin.csv')
    X = data.drop(columns=['target'])
    y = data['target']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    history: cb.History = model_report.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=val_split)
    history_df: pd.DataFrame = pd.DataFrame(history.history)

    fig, axes = plt.subplots(1, 2, figsize=(15,7))

    sns.lineplot(data=history_df['accuracy'], ax=axes[0])
    axes[0].set_title('val_ccuracy')

    sns.lineplot(data=history_df['loss'], ax=axes[1])
    axes[1].set_title('val_loss')

    model_report.evaluate(X_test, y_test)

    return model_report

In [50]:
def simulate_neural_network(data_tensor, weight_bias_tensor):

    size: int = data_tensor.shape[0]
    size_params: int = data_tensor.shape[1]

    weights: np.ndarray = weight_bias_tensor[0]
    bias: np.ndarray = weight_bias_tensor[1]
    sigmoid_tensor: list = []

    output_tensor = np.zeros((size, size_params))

    for i in range(size):
        for j in range(size_params):
            output_tensor[i, j] += (data_tensor[i, j] * weights[j])
        r = sum(output_tensor[i]) + bias[0]
        sigmoid_tensor.append(np.round(1 / (1 + np.exp(-r)), 3))



    return output_tensor, sigmoid_tensor

In [82]:
def simulate_neural_network_np(data_tensor, weight_bias_tensor):
    output_tensor = data_tensor * weight_bias_tensor[0]
    bias: float = weight_bias_tensor[1][0]
    result = output_tensor.sum(axis=1).reshape(-1, 1)
    sigmoid_tensor: np.ndarray = np.array(list(map(lambda r: np.round(1 / (1 + np.exp(-(r + bias))), 3), result)))
    
    return sigmoid_tensor
    

In [83]:
# Инициализация входных данных и весов
data_tensor = X.values
weight_tensor = [[0.29, 0.91, -1.29],[-0.01]]

# Симуляция поведения нейронного слоя
output_tensor, sigmoid = simulate_neural_network(data_tensor, weight_tensor)
output_tensor_np = simulate_neural_network_np(data_tensor, weight_tensor)

print("Входные данные:")
print(data_tensor)
print("\nТензор весов:")
print(weight_tensor)
print("\nВыход после симуляции:")
print(output_tensor)
print("\nВыход после симуляции NP:")
print(output_tensor_np)
print("\nВыход sigmoid:")
print(sigmoid)

Входные данные:
[[0 0 0]
 [0 0 1]
 [0 1 0]
 [0 1 1]
 [1 0 0]
 [1 0 1]
 [1 1 0]
 [1 1 1]]

Тензор весов:
[[0.29, 0.91, -1.29], [-0.01]]

Выход после симуляции:
[[ 0.    0.    0.  ]
 [ 0.    0.   -1.29]
 [ 0.    0.91  0.  ]
 [ 0.    0.91 -1.29]
 [ 0.29  0.    0.  ]
 [ 0.29  0.   -1.29]
 [ 0.29  0.91  0.  ]
 [ 0.29  0.91 -1.29]]

Выход после симуляции NP:
[[0.498]
 [0.214]
 [0.711]
 [0.404]
 [0.57 ]
 [0.267]
 [0.767]
 [0.475]]

Выход sigmoid:
[0.498, 0.214, 0.711, 0.404, 0.57, 0.267, 0.767, 0.475]


In [12]:
model = create_model()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ output (Dense)                  │ (None, 1)              │             4 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4 (16.00 B)

 Trainable params: 4 (16.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
create_error_report(model)

In [38]:
model.get_layer('output').get_weights()

[array([[ 0.29375646],
        [ 0.91396093],
        [-1.2950524 ]], dtype=float32),
 array([-0.09693205], dtype=float32)]

In [42]:
X = data_train.drop(columns=['target'])
y = data_train['target']
for line in X.values:
    print(line)

[0 0 0]
[0 0 1]
[0 1 0]
[0 1 1]
[1 0 0]
[1 0 1]
[1 1 0]
[1 1 1]
